In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = xR_egoposemodel(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp698'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp698/0.pth',
 '/workspace/2d_to_3d/apps/exp698/1.pth',
 '/workspace/2d_to_3d/apps/exp698/2.pth',
 '/workspace/2d_to_3d/apps/exp698/3.pth',
 '/workspace/2d_to_3d/apps/exp698/4.pth',
 '/workspace/2d_to_3d/apps/exp698/5.pth',
 '/workspace/2d_to_3d/apps/exp698/6.pth',
 '/workspace/2d_to_3d/apps/exp698/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	output_path = f'/dataset/data/xr_egopose_depth_sil/baseline698_{i}.json'
	main(model_path=p,output_path=output_path)

06:03:47 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 167.34027099609375, 'UpperStretching': 172.5648193359375, 'Walking': 164.9041748046875, 'LowerStretching': 178.85211181640625, 'Gesticuling': 117.99382781982422, 'Gaming': 142.28079223632812, 'Reacting': 140.6888427734375, 'Patting': 135.32891845703125, 'Greeting': 115.26620483398438, 'Talking': 111.90556335449219}, 'UpperBody': {'All': 137.23582458496094, 'UpperStretching': 150.60699462890625, 'Walking': 120.69486236572266, 'LowerStretching': 117.83885192871094, 'Gesticuling': 112.88623809814453, 'Gaming': 119.14292907714844, 'Reacting': 124.36115264892578, 'Patting': 124.0906753540039, 'Greeting': 107.81401062011719, 'Talking': 107.48957824707031}, 'LowerBody': {'All': 197.4447021484375, 'UpperStretching': 194.5226287841797, 'Walking': 209.11346435546875, 'LowerStretching': 239.86537170410156, 'Gesticuling': 123.10140228271484, 'Gaming': 165.418701171875, 'Reacting': 157.0165252685547, 'Patting': 146.56716918945312, 'Greeting': 122.71839141845703, 'Talking': 116.

  7%|▋         | 400/5352 [02:25<28:17,  2.92it/s]

{'FullBody': {'All': 169.1171112060547, 'UpperStretching': 173.3673095703125, 'Walking': 167.47947692871094, 'LowerStretching': 182.40338134765625, 'Gesticuling': 119.5942153930664, 'Gaming': 139.87884521484375, 'Reacting': 142.30665588378906, 'Patting': 138.8557891845703, 'Greeting': 114.63483428955078, 'Talking': 112.4413833618164}, 'UpperBody': {'All': 137.31687927246094, 'UpperStretching': 150.4978790283203, 'Walking': 122.67889404296875, 'LowerStretching': 118.0055160522461, 'Gesticuling': 114.41849517822266, 'Gaming': 113.98265838623047, 'Reacting': 127.61393737792969, 'Patting': 122.81769561767578, 'Greeting': 106.41668701171875, 'Talking': 105.06967163085938}, 'LowerBody': {'All': 200.9173583984375, 'UpperStretching': 196.2367401123047, 'Walking': 212.28004455566406, 'LowerStretching': 246.80126953125, 'Gesticuling': 124.76990509033203, 'Gaming': 165.77505493164062, 'Reacting': 156.99937438964844, 'Patting': 154.89390563964844, 'Greeting': 122.85298919677734, 'Talking': 119.813

 11%|█         | 600/5352 [03:34<27:09,  2.92it/s]

...

 93%|█████████▎| 5000/5352 [28:51<02:04,  2.83it/s]

{'FullBody': {'All': 169.83349609375, 'Patting': 138.28245544433594, 'LowerStretching': 182.05116271972656, 'UpperStretching': 174.25692749023438, 'Walking': 168.3417510986328, 'Reacting': 145.96678161621094, 'Talking': 110.88528442382812, 'Greeting': 107.74730682373047, 'Gaming': 136.2504425048828, 'Gesticuling': 117.62891387939453}, 'UpperBody': {'All': 137.8730926513672, 'Patting': 120.13275909423828, 'LowerStretching': 118.92045593261719, 'UpperStretching': 150.8380889892578, 'Walking': 123.24640655517578, 'Reacting': 127.54183959960938, 'Talking': 105.289306640625, 'Greeting': 100.47982788085938, 'Gaming': 111.28197479248047, 'Gesticuling': 110.76306915283203}, 'LowerBody': {'All': 201.79388427734375, 'Patting': 156.43214416503906, 'LowerStretching': 245.18186950683594, 'UpperStretching': 197.67578125, 'Walking': 213.43711853027344, 'Reacting': 164.3917236328125, 'Talking': 116.48125457763672, 'Greeting': 115.0147933959961, 'Gaming': 161.21890258789062, 'Gesticuling': 124.49475860

 97%|█████████▋| 5200/5352 [30:00<00:53,  2.82it/s]

{'FullBody': {'All': 169.78976440429688, 'Patting': 137.82582092285156, 'LowerStretching': 181.94786071777344, 'UpperStretching': 174.2516632080078, 'Walking': 168.26702880859375, 'Reacting': 146.28158569335938, 'Talking': 110.67044067382812, 'Greeting': 108.03372955322266, 'Gaming': 136.0131072998047, 'Gesticuling': 117.57624816894531}, 'UpperBody': {'All': 137.88754272460938, 'Patting': 119.9731216430664, 'LowerStretching': 118.89804077148438, 'UpperStretching': 150.8650665283203, 'Walking': 123.27123260498047, 'Reacting': 127.77325439453125, 'Talking': 105.13520050048828, 'Greeting': 100.94869995117188, 'Gaming': 110.9995346069336, 'Gesticuling': 110.64518737792969}, 'LowerBody': {'All': 201.69200134277344, 'Patting': 155.67849731445312, 'LowerStretching': 244.9976806640625, 'UpperStretching': 197.63824462890625, 'Walking': 213.26284790039062, 'Reacting': 164.7899169921875, 'Talking': 116.20568084716797, 'Greeting': 115.11876678466797, 'Gaming': 161.0266876220703, 'Gesticuling': 124

100%|██████████| 5352/5352 [30:53<00:00,  2.89it/s]
10:11:23 [INFO] Main: Saving evaluation results...
10:11:23 [INFO] Main: Done.
